In [ ]:
import bcrypt
import pickle


class User():

  '''Класс для информации о пользователе'''

  def __init__(self, user_id, name, surname, password):
    self.user_id = user_id
    self.name = name
    self.surname = surname
    self.set_password(password)

  def set_password(self, password):
          '''Хэширование пароля'''
          self.password_hash = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

  def user_info(self):
          '''Информация об аккаунте'''
          return f"Аккаунт: {self.name}, {self.surname}, ID: {self.user_id}"


class TransactionHistory:
    '''Класс для хранения истории транзакций пользователя'''

    def __init__(self):
        self.transactions = []

    def add_transaction(self, type_trans, amount):
        '''Добавление новой транзакции в историю'''

        self.transactions.append((type_trans,amount))

    def get_history(self):
        return self.transactions



class UserBalance():

  '''
  Класс для баланса пользователя. Состояние баланса.
  Пополнение и списание баланса. Возможность просмотра баланса
  '''

    def __init__(self, user_id, balance):
        self.user_id = user_id
        self.__balance = balance
        self.history = TransactionHistory()


    def balance_position(self, amount):

      if amount > 0:
        self.__balance += amount
        self.history.add_transaction('Пополнение', amount)
        print(f'Пополнение на сумму: {amount}')

      else:
        print('Сумма должна быть больше 0')


    def write_off(self, amount):

      if 0 < amount <= self.__balance:
        self.__balance -= amount
        self.history.add_transaction('Снятие', amount)
        print(f'Снятие на сумму: {amount}')
      else:
        print('Недостаточно средств или сумма некорректна')


    def user_info(self):
        '''Информация о балансе'''
        return f"Баланс пользователя с ID {self.user_id}: {self.__balance}"



class ml_model():
    '''
    Класс для прогнозирования цены недвижимости на основе ML-модели:
    Входные параметры: количество комнат, квадратные метры, расстояние до метро
    Таргет: цена недвижимости

    '''
    def __init__(self, pickle_filename):
        '''Загрузка обученной модели'''
      with open(pickle_filename, 'rb') as file:
        self.model = pickle.load(file)


    def data_check(self):
         '''Проверка корректности данных'''
      all_columns = ['rooms', 'square_meters', 'distance']
      if not all(col in data.columns for col in all_columns):
          raise ValueError(f"Данные должны содержать следующие столбцы: {all_columns}")


    def predict_price(self, rooms, distance, square_meters):
        '''Вывод прогноза'''
      features = {
            'rooms': [rooms],
            'distance': [distance],
            'square_meters': [square_meters]
        }
      X = pd.DataFrame(features)
      prediction = self.model.predict(X)[0]
      return prediction


class PredictionHystory():
    '''Класс для хранения истории предсказаний модели'''

    def __init__(self):
        self.history = []

    def add_prediction(self, rooms, distance, square_meters, prediction):
        '''Добавление предсказания в историю'''
        self.history.append({
            'rooms': rooms,
            'distance': distance,
            'square_meters': square_meters,
            'prediction': prediction
        })

    def show_history(self):
        '''Отображает всю историю предсказаний'''
        if not self.history:
            print("История предсказаний отсутствует")
        else:
          for i in self.history:
            print(f"Комнаты: {i['rooms']}, Площадь: {i['square_meters']}, "
                  f"Расстояние до метро: {i['distance']}, "
                  f"Предсказанная цена: {i['prediction']}")



class Admin():
  '''Роль администратора подразумевает возможность пополнять баланс пользователям
  (модерировать пополнения от пользователей) и просматривать все транзакции'''
  def __init__(self):
    self.transactions = []

  def increase_balance(self, user_balance: UserBalance, amount):
        '''Пополняет баланс пользователя и записывает транзакцию'''
        if amount > 0:
            user_balance.balance_position(amount)
            self.transactions.append({
                'user_id': user_balance.user_id,
                'amount': amount
            })
            print(f'Баланс пользователя {user_balance.user_id} пополнен на сумму: {amount}')
        else:
            print('Сумма должна быть больше 0')

    def view_transactions(self):
        '''Просмотр всех транзакций'''
        if not self.transactions:
            print("Нет транзакций.")
        else:
          for transaction in self.transactions:
            print(f"Пользователь ID: {transaction['user_id']}, Сумма: {transaction['amount']}, Тип: {transaction['type']}")

